# Statistics to Use (Ideas)
- Political - ?
- Population Density
- Migration patterns
- Age demographics
- Culture - ?
- Interest Rates
- GDP
- Number of housing units
- Proximity to social/metro centres
- City Age -- History (how would we quantify?) 
- Imports/Exports - ?
- Main Industry/Sector - Resources, raw materials
- Average Income
- Number of Schools, Education
- Geographic Location
- Number of highways, transporation infrastructure
- Terrain 

## Economic
Interest Rates, GDP, Imports/Exports, Average Income

## Important
- <b>Politics</b>
    - Partisan segration, polarity
- <b>Education</b>
- Abundance of housing units - ?
- City Age/History
- <b>Main Industry Sector</b>
- <b> Economic Measures</b>
- <b>Geographic Location</b>
    - Proximity to other cities, metro centres
- Number of highways, transporation infrastructure

# TO-DO:
- Add more features, fix poverty data

In [20]:
# Import Necessary Libraries
import numpy as np 
import pandas as pd

# FEATURE 1: RATIO OF PEOPLE TAKING PUBLIC TRANSPORATION TO WORK

transportation_data = pd.read_csv('../datasets/transportation_data/ACSST5Y2010.S0802_data_with_overlays_2020-03-04T134515.csv')
transportation_data.NAME = transportation_data.NAME.map(lambda x: x.replace(' Metro Area', '').replace(' Micro Area', ''))


# Filter data to get only appropriate metropolitan areas
transportation_data = transportation_data[~transportation_data.NAME.str.contains('United State')]
num_public_transport = transportation_data['S0802_C04_001E'].to_numpy()[1:].astype(int)
num_workers = transportation_data['S0802_C01_001E'].to_numpy()[1:].astype(int)
ratio = np.divide(num_public_transport, num_workers)

# Standardisation, Feature Scaling
ratio = (ratio - np.mean(ratio))/(ratio.std())
ratio = [float(r) for r in ratio]

# Create new DataFrame with each metro area and corresponding attribute values
new_df = pd.DataFrame(zip(transportation_data['NAME'].to_numpy()[1:],  ratio), columns = ['metro_name', 'ratio_of_public_transport']).set_index('metro_name').sort_values('metro_name')
new_df['ratio_of_public_transport'].fillna(0, inplace = True)
new_df

,ratio_of_public_transport
metro_name,
"Abbeville, LA",-0.355924
"Aberdeen, SD",-0.362291
"Aberdeen, WA",0.248173
"Abilene, TX",-0.332337
"Ada, OK",-0.520086
...,...
"York-Hanover, PA",0.002138
"Youngstown-Warren-Boardman, OH-PA",-0.186601
"Yuba City, CA",-0.016467


In [21]:
# Standardise a pandas Series to prepare for K-Means
def standardise(df, series):
    df = df.copy(deep = False)
    column = df.loc[:, series]
    standardised = (column - column.mean()) / column.std()
    return standardised

# Standardise when converted to array
def standardise_array(arr):
    return (arr - arr.mean()) / arr.std()

In [22]:

# FEATURE 2: MEDIAN HOUSEHOLD INCOME

income_data = pd.read_csv('../datasets/income_data/ACSST5Y2010.S2503_data_with_overlays_2020-03-04T163518.csv')
income_data = income_data[~income_data.NAME.str.contains('United States')]
income_data.NAME = income_data.NAME.map(lambda x: x.replace(' Metro Area', '').replace(' Micro Area', ''))
median_income = income_data['S2503_C01_013E'].to_numpy()[1:].astype(int)

# Standardise
median_income = standardise_array(median_income)

# Make sure every element of the median_income column is a float
new_df['median_income'] = [float(m) for m in median_income]
new_df

,ratio_of_public_transport,median_income
metro_name,,
"Abbeville, LA",-0.355924,-0.622420
"Aberdeen, SD",-0.362291,1.245645
"Aberdeen, WA",0.248173,0.120348
"Abilene, TX",-0.332337,0.407350
"Ada, OK",-0.520086,0.195562
...,...,...
"York-Hanover, PA",0.002138,0.532464
"Youngstown-Warren-Boardman, OH-PA",-0.186601,0.268693
"Yuba City, CA",-0.016467,0.193791


In [23]:
# FEATURE 3: PERCENT BELOW POVERTY LEVEL

#### CURRENTLY NOT WORKING #####

poverty_data = pd.read_csv('../datasets/poverty_data/ACSST1Y2010.S1701_data_with_overlays_2020-03-04T183351.csv')
poverty_data = poverty_data[~poverty_data.NAME.str.contains('United States')]
poverty_data.NAME = poverty_data.NAME.map(lambda x: x.replace(' Metro Area', '').replace(' Micro Area', ''))
percentages = poverty_data['S1701_C03_001E'].to_numpy()[1:].astype(float)
percentages = standardise_array(percentages)

poverty_df = pd.DataFrame(zip(poverty_data['NAME'][1:], percentages),
             columns = ['metro_name', 'pctg_below_poverty_line']).sort_values('metro_name')

new_df2 = new_df.merge(poverty_df, on = 'metro_name', how='outer', suffixes=('', '_y')).fillna(0)
new_df2                 

,metro_name,ratio_of_public_transport,median_income,pctg_below_poverty_line
0,"Abbeville, LA",-0.355924,-0.622420,0.000000
1,"Aberdeen, SD",-0.362291,1.245645,0.000000
2,"Aberdeen, WA",0.248173,0.120348,0.013290
3,"Abilene, TX",-0.332337,0.407350,0.139864
4,"Ada, OK",-0.520086,0.195562,0.000000
...,...,...,...,...
950,"York-Hanover, PA",0.002138,0.532464,-1.252450
951,"Youngstown-Warren-Boardman, OH-PA",-0.186601,0.268693,-0.002531
952,"Yuba City, CA",-0.016467,0.193791,0.345547
953,"Yuma, AZ",0.635026,-0.604710,0.472121


In [40]:
# FEATURE 4: POPULATION OF PEOPLE 3 AND OVER ENROLLED IN SCHOOL

school_data = pd.read_csv('../datasets/school_data/ACSST5Y2010.S1401_data_with_overlays_2020-03-04T192940.csv')
school_data.NAME = school_data.NAME.map(lambda x: x.replace(' Metro Area', '').replace(' Micro Area', ''))

# Log-transform, standardise
enrolled = school_data['S1401_C01_001E'].to_numpy()[1:].astype(int)
enrolled = standardise_array(enrolled)

# Correct type, save to test DataFrame
new_df2['enrolled_in_school'] = [float(e) for e in enrolled]
new_df2.to_csv('test_01.csv')
new_df2

,metro_name,ratio_of_public_transport,median_income,pctg_below_poverty_line,enrolled_in_school
0,"Abbeville, LA",-0.355924,-0.622420,0.000000,0.035869
1,"Aberdeen, SD",-0.362291,1.245645,0.000000,0.346162
2,"Aberdeen, WA",0.248173,0.120348,0.013290,-0.077303
3,"Abilene, TX",-0.332337,0.407350,0.139864,0.455581
4,"Ada, OK",-0.520086,0.195562,0.000000,-0.228167
...,...,...,...,...,...
950,"York-Hanover, PA",0.002138,0.532464,-1.252450,-0.257513
951,"Youngstown-Warren-Boardman, OH-PA",-0.186601,0.268693,-0.002531,0.005607
952,"Yuba City, CA",-0.016467,0.193791,0.345547,-0.255924
953,"Yuma, AZ",0.635026,-0.604710,0.472121,-0.217837


In [41]:
cbsa = pd.read_csv('../datasets/cbsa.csv')[['CBSA_ID', 'NAME', 'SQMI', 'HSE_UNITS', 'AVE_FAM_SZ', 'POP10_SQMI']]

for col in cbsa.columns[2:]:
    cbsa[col] = standardise(cbsa, col)

new_df3 = new_df2.merge(cbsa, left_on = 'metro_name', right_on = 'NAME', how = 'inner')
new_df3 = new_df3.dropna().drop(columns = ['NAME'])
new_df3

,metro_name,ratio_of_public_transport,median_income,pctg_below_poverty_line,enrolled_in_school,CBSA_ID,SQMI,HSE_UNITS,AVE_FAM_SZ,POP10_SQMI
0,"Aberdeen, SD",-0.362291,1.245645,0.000000,0.346162,10100,0.419313,-0.265327,0.373006,-0.551088
1,"Aberdeen, WA",0.248173,0.120348,0.013290,-0.077303,10140,0.028771,-0.207854,0.414232,-0.426037
2,"Abilene, TX",-0.332337,0.407350,0.139864,0.455581,10180,0.368355,-0.087447,0.480195,-0.307371
3,"Ada, OK",-0.520086,0.195562,0.000000,-0.228167,10220,-0.465152,-0.272564,0.455459,-0.351006
4,"Adrian, MI",-0.424802,0.868949,-0.429719,-0.224541,10300,-0.450319,-0.178981,0.463704,0.072041
...,...,...,...,...,...,...,...,...,...,...
794,"York-Hanover, PA",0.002138,0.532464,-1.252450,-0.257513,49620,-0.389084,0.292187,0.447214,1.915353
795,"Youngstown-Warren-Boardman, OH-PA",-0.186601,0.268693,-0.002531,0.005607,49660,-0.047236,0.574632,0.414232,1.099591
796,"Yuba City, CA",-0.016467,0.193791,0.345547,-0.255924,49700,-0.248958,-0.116118,0.818251,0.083215
797,"Yuma, AZ",0.635026,-0.604710,0.472121,-0.217837,49740,1.500424,-0.024277,0.785270,-0.437744


In [45]:
political_data = pd.read_csv('../geographic_eda/pol_percents_DR.csv')
political_data

,CBSACode,democrat,green,republican,total,%democrat,%republican
0,10100,422637.0,4661.0,571148.0,993785.0,0.425280,0.574720
1,10140,72221.0,1059.0,62181.0,134402.0,0.537351,0.462649
2,10180,231887.0,1619.0,515957.0,747844.0,0.310074,0.689926
3,10220,36182.0,84.0,90559.0,126741.0,0.285480,0.714520
4,10300,102318.0,760.0,117362.0,219680.0,0.465759,0.534241
...,...,...,...,...,...,...,...
900,49660,1372358.0,12786.0,1037424.0,2409782.0,0.569495,0.430505
901,49700,92418.0,1101.0,151179.0,243597.0,0.379389,0.620611
902,49740,94286.0,555.0,128224.0,222510.0,0.423738,0.576262
903,49780,75690.0,882.0,104115.0,179805.0,0.420956,0.579044


In [52]:
new_df4 = new_df3.merge(political_data, left_on = 'CBSA_ID', right_on = 'CBSACode', how = 'left').drop(columns = ['democrat', 'green', 'republican', 'total'])
new_df4

,metro_name,ratio_of_public_transport,median_income,pctg_below_poverty_line,enrolled_in_school,CBSA_ID,SQMI,HSE_UNITS,AVE_FAM_SZ,POP10_SQMI,CBSACode,%democrat,%republican
0,"Aberdeen, SD",-0.362291,1.245645,0.000000,0.346162,10100,0.419313,-0.265327,0.373006,-0.551088,10100.0,0.425280,0.574720
1,"Aberdeen, WA",0.248173,0.120348,0.013290,-0.077303,10140,0.028771,-0.207854,0.414232,-0.426037,10140.0,0.537351,0.462649
2,"Abilene, TX",-0.332337,0.407350,0.139864,0.455581,10180,0.368355,-0.087447,0.480195,-0.307371,10180.0,0.310074,0.689926
3,"Ada, OK",-0.520086,0.195562,0.000000,-0.228167,10220,-0.465152,-0.272564,0.455459,-0.351006,10220.0,0.285480,0.714520
4,"Adrian, MI",-0.424802,0.868949,-0.429719,-0.224541,10300,-0.450319,-0.178981,0.463704,0.072041,10300.0,0.465759,0.534241
...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,"York-Hanover, PA",0.002138,0.532464,-1.252450,-0.257513,49620,-0.389084,0.292187,0.447214,1.915353,49620.0,0.423997,0.576003
795,"Youngstown-Warren-Boardman, OH-PA",-0.186601,0.268693,-0.002531,0.005607,49660,-0.047236,0.574632,0.414232,1.099591,49660.0,0.569495,0.430505
796,"Yuba City, CA",-0.016467,0.193791,0.345547,-0.255924,49700,-0.248958,-0.116118,0.818251,0.083215,49700.0,0.379389,0.620611
797,"Yuma, AZ",0.635026,-0.604710,0.472121,-0.217837,49740,1.500424,-0.024277,0.785270,-0.437744,49740.0,0.423738,0.576262


In [79]:
# Get Lat + Long data
geocoding = pd.read_csv('../geocoding_data/geocode_cbsa_crosswalk.csv')
final_df = new_df4.merge(geocoding, left_on = 'CBSA_ID', right_on = 'CBSA', how = 'inner').drop_duplicates('CBSA_ID')
final_df = final_df.drop(columns = ['Unnamed: 0', 'CBSACode', 'CBSA'])

# Standardise remaining columns
final_df["%democrat"] = standardise(final_df, "%democrat")
final_df["%republican"] = standardise(final_df, "%republican")
final_df["Latitude"] = standardise(final_df, "Latitude")
final_df["Longitude"] = standardise(final_df, "Longitude")
final_df

,metro_name,ratio_of_public_transport,median_income,pctg_below_poverty_line,enrolled_in_school,cbsa_id,sqmi,hse_units,avg_fam_sz,pop_sqmi,pct_dem,pct_repub,latitude,longitude
0,"Aberdeen, SD",-0.362291,1.245645,0.000000,0.346162,10100,0.419313,-0.265327,0.373006,-0.551088,-0.006796,0.006796,1.361289,-0.459699
24,"Aberdeen, WA",0.248173,0.120348,0.013290,-0.077303,10140,0.028771,-0.207854,0.414232,-0.426037,1.023474,-1.023474,1.635780,-2.312895
46,"Abilene, TX",-0.332337,0.407350,0.139864,0.455581,10180,0.368355,-0.087447,0.480195,-0.307371,-1.065892,1.065892,-1.025447,-0.496142
79,"Ada, OK",-0.520086,0.195562,0.000000,-0.228167,10220,-0.465152,-0.272564,0.455459,-0.351006,-1.291988,1.291988,-0.590006,-0.327322
87,"Adrian, MI",-0.424802,0.868949,-0.429719,-0.224541,10300,-0.450319,-0.178981,0.463704,0.072041,0.365331,-0.365331,0.758064,0.617013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20764,"York-Hanover, PA",0.002138,0.532464,-1.252450,-0.257513,49620,-0.389084,0.292187,0.447214,1.915353,-0.018590,0.018590,0.405970,1.118538
20816,"Youngstown-Warren-Boardman, OH-PA",-0.186601,0.268693,-0.002531,0.005607,49660,-0.047236,0.574632,0.414232,1.099591,1.318976,-1.318976,0.573224,0.896621
20911,"Yuba City, CA",-0.016467,0.193791,0.345547,-0.255924,49700,-0.248958,-0.116118,0.818251,0.083215,-0.428676,0.428676,0.130559,-2.139824
20943,"Yuma, AZ",0.635026,-0.604710,0.472121,-0.217837,49740,1.500424,-0.024277,0.785270,-0.437744,-0.020970,0.020970,-0.929281,-1.554558


In [112]:
# Get AU3 results
au3_results = pd.read_csv('../AU3_results.csv')

# Filter
au3_results = au3_results[au3_results['CBSA_Codes'] != 'United_States']
au3_results.CBSA_Codes = au3_results.CBSA_Codes.astype(np.float64)
final_df_au3 = final_df.merge(au3_results, left_on = 'cbsa_id', right_on = 'CBSA_Codes', how = 'inner')
final_df_au3 = final_df_au3.drop(columns = ['Unnamed: 0', 'CBSA_Codes'])

# Standardise AU3 output
final_df_au3['AU3'] = standardise(final_df_au3, 'AU3')

# Reformat titles
final_df_au3.columns = ['metro_name', 'ratio_of_public_transport', 'median_income', 'pctg_below_poverty_line', 'enrolled_in_school', 'cbsa_id', 'sqmi', 'hse_units', 'avg_fam_sz', 'pop_sqmi', 'pct_dem', 'pct_repub', 'latitude', 'longitude', 'au3']

# Reorder columns
final_df_au3 = final_df_au3[['metro_name', 'cbsa_id', 'ratio_of_public_transport', 'median_income', 'pctg_below_poverty_line', 'enrolled_in_school', 'sqmi', 'hse_units', 'avg_fam_sz', 'pop_sqmi', 'pct_dem', 'pct_repub', 'latitude', 'longitude', 'au3']]
final_df_au3

,metro_name,cbsa_id,ratio_of_public_transport,median_income,pctg_below_poverty_line,enrolled_in_school,sqmi,hse_units,avg_fam_sz,pop_sqmi,pct_dem,pct_repub,latitude,longitude,au3
0,"Aberdeen, WA",10140.0,0.248173,0.120348,0.013290,-0.077303,0.028771,-0.207854,0.414232,-0.426037,1.023474,-1.023474,1.635780,-2.312895,0.375230
1,"Ada, OK",10220.0,-0.520086,0.195562,0.000000,-0.228167,-0.465152,-0.272564,0.455459,-0.351006,-1.291988,1.291988,-0.590006,-0.327322,-1.589582
2,"Adrian, MI",10300.0,-0.424802,0.868949,-0.429719,-0.224541,-0.450319,-0.178981,0.463704,0.072041,0.365331,-0.365331,0.758064,0.617013,0.583803
3,"Akron, OH",10420.0,0.191674,0.533610,-0.255679,-0.264682,-0.383568,0.758794,0.447214,3.422888,1.218050,-1.218050,0.604046,0.784054,0.255774
4,"Albany, GA",10500.0,0.001436,1.938825,1.674574,-0.159467,-0.102800,-0.100204,0.546157,-0.198816,-0.053201,0.053201,-1.174070,0.589454,0.259028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,"York-Hanover, PA",49620.0,0.002138,0.532464,-1.252450,-0.257513,-0.389084,0.292187,0.447214,1.915353,-0.018590,0.018590,0.405970,1.118538,0.454380
597,"Youngstown-Warren-Boardman, OH-PA",49660.0,-0.186601,0.268693,-0.002531,0.005607,-0.047236,0.574632,0.414232,1.099591,1.318976,-1.318976,0.573224,0.896621,0.294844
598,"Yuba City, CA",49700.0,-0.016467,0.193791,0.345547,-0.255924,-0.248958,-0.116118,0.818251,0.083215,-0.428676,0.428676,0.130559,-2.139824,1.016019
599,"Yuma, AZ",49740.0,0.635026,-0.604710,0.472121,-0.217837,1.500424,-0.024277,0.785270,-0.437744,-0.020970,0.020970,-0.929281,-1.554558,0.735752


In [113]:
# SAVE FINAL VECTORS TO .CSV
final_df_au3.to_csv('../geographic_eda/clustering_vectors.csv', index = False, header = True)

## Final Vector
- 13 components